### In this notebook

- Reading some expression data into a *pandas* `DataFrame`.
- Looking up expression data for a specific gene or sample, resulting in a *pandas* `Series`.
- Doing some slicing to select a specific range of samples.
- Selecting all genes that start with "A" (because why not?)
- Merging two dataframes
- Additional resources

### Introduction

*pandas* is a very powerful package that provides a lot of functions for working with *labeled data* (think spreadsheets). It provides two key data structures that we will come across quite a lot in the workshop:

1. `pandas.DataFrame`: A `DataFrame` represents tabular data, i.e. data you could also store in an Excel spreadsheet. Each `DataFrame` has row labels (accessible using the `DataFrame.index` attribute), and column labels (accessible using the `DataFrame.columns`) attribute, in addition to the data itself (accessible using the `DataFrame.data` attribute).

2. `pandas.Series`: A `Series` represents data in the form of a vector (i.e., an ordered list of elements), where each element has a label. For example, a `Series` is what you get when you select a single column from a `DataFrame`. Consequently, a `Series` also contains "row labels", accessible through the `Series.index` attribute. We can use use a `Series` to store the expression profile of a single sample.

*pandas* stores the data itself (without the labels) in matrix form, using *NumPy* arrays. It's fine if you ignore this piece of information for now.

### 1. Reading expression data from a tab-delimited text file

We start by reading expression data from a tab-delimited text file. All the expression data we're using in the workshop examples are in the following format:

```
Genes    SampleA   SampleB   SampleB   ...
GeneA    2.51521   3.51411   4.12512   ...
GeneB    7.32312   1.51262   3.12512   ...
GeneC    4.21256   4.12632   3.16212   ...
...      ...       ...       ...       ...
```

Before I started using `pandas`, reading a data file like this was a rather tedious and repetitive exercise. But with *pandas* we're in a much better place, because we can essentially do this one line:

In [22]:
import os
import pandas as pd  # renaming "pandas" to "pd" is a convention

# this is some of the example data for this workshop
# we're using os.path.join() so that the resulting path is correctly formatted on both Windows and Linux/Mac
expression_file = os.path.join('..', 'data', 'brca_expression_5yr_dead.tsv')
print('Path to expression file:', expression_file)

df = pd.read_csv(expression_file, sep='\t', header=0, index_col=0)
# another convention is to call the variable that holds your main DataFrame object "df"

print('Matrix shape:', df.shape)  # prints (x,y), where x is the number of rows, and y the number of columns
print('Genes:', df.index.tolist()[:5])  # prints the first five genes (rows) as a list
print('Samples:', df.columns.tolist()[:5])  # prints the first five samples (columns) as a list

Path to expression file: ../data/brca_expression_5yr_dead.tsv
Matrix shape: (19785, 30)
Genes: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2']
Samples: ['TCGA-BH-A1EV-01A', 'TCGA-B6-A0WW-01A', 'TCGA-LL-A73Z-01A', 'TCGA-BH-A18J-01A', 'TCGA-A2-A04P-01A']


### 2. Looking up expression data for a particular sample or gene

One of the key strengths of the *pandas* `DataFrame` (and one of the first things to learn), is the different ways in which you can access exactly the data you need. We'll start by selecting the expression profile of a specific sample.

In [3]:
sample = 'TCGA-B6-A0WW-01A'
profile = df[sample]  # and we're done

print(profile[:3])  # prints the expression of the first three genes

Genes
A1BG    2.32193
A1CF    2.32193
A2M     8.07841
Name: TCGA-B6-A0WW-01A, dtype: float64


What's the type of `profile`? Let's check:

In [4]:
print(type(profile))

<class 'pandas.core.series.Series'>


It's a `pandas.Series` object. The `index` of the series corresponds to the genes. Note also that the series automatically got a `name` corresponding to the sample label. Note also that above we've used the basic slicing syntax "[:3]" to print the first three elements of the series. This is the same syntax used to select ranges of elements in simple Python lists!

Now we're going to access expression values of a single gene, across all samples:

In [5]:
gene = 'A2M'
gene_expression = df.loc[gene]  # done!

print(gene_expression[:3])
print()
print('Type:', type(gene_expression))

TCGA-BH-A1EV-01A    7.66424
TCGA-B6-A0WW-01A    8.07841
TCGA-LL-A73Z-01A    9.37704
Name: A2M, dtype: float64

Type: <class 'pandas.core.series.Series'>


Note that the result is again a `Series` object, but this time its index consists of sample labels, and the series `name` corresponds to the gene we selected.



Finally, just know that the .loc indexing method also allows you to select samples, like this:

In [6]:
sample = 'TCGA-B6-A0WW-01A'
profile2 = df.loc[:, sample]

# check to see if we're selecting the same thing as with the "df[sample]" syntax
print(profile.equals(profile2))  # True

True


The `[:, sample]` syntax is the same as the one used for NumPy arrays, but pandas allows you to use strings to refer to a specific row or column by their label.

### 3. Slicing to select a specific range of samples

Remember that we have 30 samples in our dataset. Let's say we only want to select the data for the first 10 of them. How would we do that? In addition to `.loc`, there's the `.iloc` operator that allows you to refer to rows and columns by their indices, and select specific ranges (aka. slicing):

In [7]:
first_samples = df.iloc[:, :10]  # the first 10 samples

print(first_samples.shape)

(19785, 10)


Again, all the slicing rules from NumPy apply. For example:

In [8]:
last_samples = df.iloc[:, -10:]  # the last 10 samples

print(last_samples.shape)

(19785, 10)


### 4. Select all the genes that start with "A"

Just to show another example of the many different things a *pandas* `DataFrame` allows you to do, we are going to select all the genes that start with "A".

First, let's find out how many genes start with "A". Remember that the genes serve as the (row) index for our `DataFrame`, which is stored in `DataFrame.index`.

In [9]:
sel = df.index.str.startswith('A')

print(type(sel))
print('Number of genes starting with "A":', sel.sum())

<class 'numpy.ndarray'>
Number of genes starting with "A": 1592


Let's now select our data:

In [10]:
A_only = df.loc[sel]  # done

print(A_only.shape)

(1592, 30)


What we've done here is to first create a Boolean array (`sel`), and then we've used that array to select the genes we're interested in. This example shows that the `.loc` operator not only accepts strings (row or columns labels), but also Boolean arrays. This provdes a very useful way of selecting samples based on arbitrary conditions.

### 5. Merge two dataframes

In the previous sections, we've looked at selecting subsets of data from a `DataFrame`. However, sometimes we would also like to merge some parts together, creating a merged `DataFrame`. The simplest function for doing that is `pandas.concat()`.

For example, let's select the genes starting with Z (as before with A), and then merge the two sets together.

In [13]:
Z_only = df.loc[df.index.str.startswith('Z')]
print(Z_only.shape)

combined = pd.concat([A_only, Z_only], axis=0)
print(combined.shape)

(800, 30)
(2392, 30)


The `axis=0` parameter is not strictly necessary, since 0 is the default value if the parameter isn't specified. However, it's important to understand that we always concatenate along a specific axis. In this case, we're concatening along the rows (genes), which is the first axis (0). Had we specified `axis=1`, we would have gotten a very different result:

In [17]:
messy = pd.concat([A_only, Z_only], axis=1)
print(messy.shape)

(2392, 60)


Note that we all of a sudden have a DataFrame with 60 columns (instead of 30). This is because we've concatenated along the columns, essentially treating the samples in `A_only` and `Z_only` as separate entities. However, since `Z_only` contains none of the genes in `A_only`, and vice versa, these values have been filled in with `NaN` (not a number):

In [19]:
print(messy[:2])

      TCGA-BH-A1EV-01A  TCGA-B6-A0WW-01A  TCGA-LL-A73Z-01A  TCGA-BH-A18J-01A  \
A1BG           2.32193           2.32193           2.32193           2.32193   
A1CF           2.32193           2.32193           2.32193           2.32193   

      TCGA-A2-A04P-01A  TCGA-AR-A0TY-01A  TCGA-AC-A62V-01A  TCGA-BH-A1FU-01A  \
A1BG           2.32193           2.32193           2.32193           2.32193   
A1CF           2.32193           2.32193           2.32193           2.32193   

      TCGA-BH-A18Q-01A  TCGA-A2-A0YT-01A        ...         TCGA-A2-A3XU-01A  \
A1BG           2.32193           2.32193        ...                      NaN   
A1CF           2.32193           2.32193        ...                      NaN   

      TCGA-A2-A0CM-01A  TCGA-A2-A3XY-01A  TCGA-BH-A1EW-01A  TCGA-BH-A0BF-01A  \
A1BG               NaN               NaN               NaN               NaN   
A1CF               NaN               NaN               NaN               NaN   

      TCGA-BH-A18L-01A  TCGA-BH-A208

### 6. Resources

- Extensive 2012 video tutorial from Wes McKinney, the creator of *pandas*: https://www.youtube.com/watch?v=w26x-z-BdWQ
- Talk on how Pandas works "behind the scenes": https://www.youtube.com/watch?v=DpyhdO4aM04
- A large number of *pandas* tutorials are available from youtube.
- The pandas documentation: http://pandas.pydata.org/pandas-docs/stable/

### Copyright and License

Copyright (c) 2016 Florian Wagner.

This work is licensed under a [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License](http://creativecommons.org/licenses/by-nc-sa/4.0/).